In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
# Load the data
train_filepath = "../input/store-sales-time-series-forecasting/train.csv"
oil_filepath = "../input/store-sales-time-series-forecasting/oil.csv"
holidays_filepath = "../input/store-sales-time-series-forecasting/holidays_events.csv"
transactions_filepath = "../input/store-sales-time-series-forecasting/transactions.csv"
stores_filepath = "../input/store-sales-time-series-forecasting/stores.csv"



In [ ]:
# Read the data
train_data = pd.read_csv(train_filepath, index_col="id")
oil_data = pd.read_csv(oil_filepath, index_col="date", parse_dates=True)
holidays_data = pd.read_csv(holidays_filepath, index_col="date", parse_dates=True)
transactions_data = pd.read_csv(transactions_filepath, index_col="date")



In [ ]:
# Convert all 'date' columns into Panda's format:
train_data['date'] = pd.to_datetime(train_data['date'], format = "%Y-%m-%d")
oil_data['date'] = pd.to_datetime(oil_data['date'], format = "%Y-%m-%d")
holidays_data['date'] = pd.to_datetime(holidays_data['date'], format = "%Y-%m-%d")
transactions_data['date'] = pd.to_datetime(transactions_data['date'], format ="%Y-%m-%d")

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
oil_data.head()


# **Data Preparation with EDAs**

The following EDA provides an overview of the data and data relationships 

In [ ]:
# Display changes in Oil Price over time
plt.figure(figsize=(16,6))
sns.lineplot(data=oil_data.dcoilwtico, label="Oil price")
plt.title('Ecuador Oil Price')
plt.xlabel('Date')
plt.ylabel('Price')

In [ ]:
plt.figure(figsize=(16,6))
sns.scatterplot(x=train_data['onpromotion'], y=train_data['sales'])
plt.title('Promotion and Sales from 2013 to 2017')
plt.xlabel('Discount Promotion')
plt.ylabel('Sales')


Prepare relevant data for Regression and Machine Learning.
Address NaN cell values.
Add dummy variable for holildays.

In [ ]:
#Merge Data
train_data_m1 = train_data.merge(oil_data, on = 'date', how = 'left')
train_data_m1

In [ ]:
train_data_m1.head()

In [ ]:
train_data_m1.tail()

In [ ]:
# Daily Oil Price
# Group the data set by date and oil price per day
oil_price = train_data_m1.groupby('date').dcoilwtico.mean()
oil_price

In [ ]:
# Daily Total Sales
# grouping data set by date and deriving ave sales per day
total_sales = train_data_m1.groupby('date').sales.sum()
total_sales

In [ ]:
nbr = train_data_m1.groupby('date').store_nbr.nunique()
nbr

In [ ]:
total_sales.to_frame()

In [ ]:
nbr.to_frame()

In [ ]:
nbr_sales = pd.concat([total_sales, nbr], axis=1)
nbr_sales

In [ ]:
nbr_sales['Ave Store Sales'] = nbr_sales['sales']/nbr_sales['store_nbr']
nbr_sales

In [ ]:
ave_nbr_sales = nbr_sales["Ave Store Sales"]
ave_nbr_sales

In [ ]:
ave_nbr_sales.to_frame()

In [ ]:
#converting data to data frame
oil_price.to_frame()

In [ ]:
# CONCAT to merge columns with similar id
oil_sales = pd.concat([ave_nbr_sales, oil_price], axis=1)
oil_sales

In [ ]:
#filling NaN with values above and below the cell
oil_sales['dcoilwtico'] = oil_sales['dcoilwtico'].fillna(method='ffill')
oil_sales['dcoilwtico'] = oil_sales['dcoilwtico'].fillna(method='bfill')
oil_sales

**Diagnostics of Variables**

In [ ]:
#Ave Sales Diagnostics

import statsmodels.api as sm
from scipy.stats import norm
import pylab

ave_nbr_sales = norm.rvs(size=1000)
sm.qqplot(ave_nbr_sales, line='45')
pylab.show()

from statsmodels.stats.diagnostic import lilliefors
ave_nbr_sales = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(ave_nbr_sales)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')
        

*Lilliefors Test for Normality*

The Lilliefors test is a normality test based on the Kolmogorov–Smirnov test. As all the above methods, this test is used to check if the data come from a normal distribution.

If the p-value ≤ 0.05, then we reject the null hypothesis i.e. we assume the distribution of our variable is not normal/gaussian.
If the p-value > 0.05, then we fail to reject the null hypothesis i.e. we assume the distribution of our variable is normal/gaussian.

In [ ]:
oil_price = oil_price.fillna(method='ffill')
oil_price = oil_price.fillna(method='bfill')
oil_price


In [ ]:
#Oil Price Diagnostics

oil_price = norm.rvs(size=1000)
sm.qqplot(oil_price, line='45')
pylab.show()

oil_price = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(oil_price)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
# add dummy to holidays

holidays_data['Dummy Holiday'] = '1'
holidays_data

In [ ]:
#"groupby" Holidays_data by Dummy Holiday

dummy_holidays = holidays_data["Dummy Holiday"]
dummy_holidays

In [ ]:
dummy_holidays.to_frame()

In [ ]:
final_df = oil_sales.merge(dummy_holidays, on = 'date', how = 'left')
final_df

In [ ]:
#replace NaN with 0

final_df['Dummy Holiday'] = final_df['Dummy Holiday'].fillna(0)
final_df

In [ ]:
# add 'on promotion' to final_df
# onpromotion has number of products on sale that  day

promotion = train_data_m1.groupby('date').onpromotion.sum()
promotion

In [ ]:
promotion.to_frame()

In [ ]:
nbr_promo = pd.concat([promotion, nbr], axis=1)
nbr_promo

In [ ]:
nbr_promo['Ave Store Promo'] = nbr_promo['onpromotion']/nbr_promo['store_nbr']
nbr_promo

In [ ]:
ave_nbr_promo = nbr_promo["Ave Store Promo"]
ave_nbr_promo

In [ ]:
ave_nbr_promo.to_frame()

In [ ]:
#Use merge to add a column of data to the set

final_df = final_df.merge(ave_nbr_promo, on = 'date', how = 'left')
final_df

In [ ]:
# Ave_promotion Diagnostics


ave_nbr_promo = norm.rvs(size=1000)
sm.qqplot(ave_nbr_promo, line='45')
pylab.show()

ave_nbr_promo = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(ave_nbr_promo)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
import numpy as np
import pylab
import scipy.stats as stats
import seaborn as sns

ave_nbr_promo = np.random.normal(loc = 20, scale = 5, size = 150)
ax = sns.distplot(ave_nbr_promo)

In [ ]:
oil_price = np.random.normal(loc = 20, scale = 5, size = 150)
ax = sns.distplot(oil_price)

In [ ]:
ave_nbr_sales = np.random.normal(loc = 20, scale = 5, size = 150)
ax = sns.distplot(ave_nbr_sales)

In [ ]:
#Regression Plot: Overall Avereage Sales and Explanatory Variables
plt.figure(figsize=(16,6))
plt.title('Average Store Sales and Products on Promotion')
sns.regplot(x=final_df['Ave Store Promo'], y=final_df['Ave Store Sales'])
plt.xlabel('Ave. Products on Promotion')
plt.ylabel('Ave Store Sales')

In [ ]:
#Regression Plot: Overall Avereage Sales and Explanatory Variables
plt.figure(figsize=(16,6))
plt.title('Average Products on Promotion and Oil Price')
sns.regplot(x=final_df['dcoilwtico'], y=final_df['Ave Store Promo'])
plt.xlabel('Oil Price')
plt.ylabel('Ave. Products on Promotion')

In [ ]:
plt.figure(figsize=(16,6))
plt.title('Average Store Sales and Oil Price')
sns.regplot(x=final_df['dcoilwtico'], y=final_df['Ave Store Sales'])
plt.xlabel('Oil Price')
plt.ylabel('Average Store Sales')

Narrow the analysis on a specific product group - Grocery Sales

In [ ]:
#Exploring the correlation of numerical features
plt.figure(figsize=(5,3), dpi=150)
sns.heatmap(final_df.corr(), annot=True)

In [ ]:
# grouping data set by date and deriving ave sales per day
product_sales = train_data.groupby('family').sales.sum()/1000000
product_sales = product_sales.sort_values(ascending=False)
product_sales.to_frame()

In [ ]:
plt.figure(figsize=(20,10))
ax = product_sales.plot.pie(x='lab', y='val', rot=0)
plt.xlabel("Product Family")
plt.title("Sales by Product Family")


In [ ]:
plt.figure(figsize=(25,13))
g = sns.PairGrid(final_df)
g.map(sns.scatterplot)

In [ ]:
#Group by multiple columns and averaging sales
family_sales = train_data_m1.groupby(["date","family"]).sales.sum()
family_sales

In [ ]:
family_sales.to_frame()

In [ ]:
#Extracting Select Data from a DF

grocery_sales = train_data_m1.loc[train_data_m1.family == 'GROCERY I']
grocery_sales

In [ ]:
# Average Grocery I Sales, grouped by date
tot_grocery_sales = grocery_sales.groupby('date').sales.sum()
tot_grocery_sales

In [ ]:
tot_grocery_sales.to_frame()

In [ ]:
nbr_gsales = pd.concat([tot_grocery_sales, nbr], axis=1)
nbr_gsales

In [ ]:
nbr_gsales['Ave Store Sales'] = nbr_gsales['sales']/nbr_gsales['store_nbr']
nbr_gsales

In [ ]:
ave_nbr_gsales = nbr_gsales["Ave Store Sales"]
ave_nbr_gsales

In [ ]:
ave_nbr_gsales.to_frame()

In [ ]:
# Average Grocery I Sales, grouped by date
oil_prices = grocery_sales.groupby('date').dcoilwtico.mean()
oil_prices

In [ ]:
oil_grocery_sales = pd.concat([ave_nbr_gsales, oil_prices], axis = 1)
oil_grocery_sales

In [ ]:
oil_grocery_sales['dcoilwtico'] = oil_grocery_sales['dcoilwtico'].fillna(method='ffill')
oil_grocery_sales['dcoilwtico'] = oil_grocery_sales['dcoilwtico'].fillna(method='bfill')
oil_grocery_sales

In [ ]:
# Merge holiday dummy data to the set

grocery_final_df = oil_grocery_sales.merge(dummy_holidays, on = 'date', how = 'left')
grocery_final_df

In [ ]:
#replace NaN with 0

grocery_final_df['Dummy Holiday'] = grocery_final_df['Dummy Holiday'].fillna(0)
grocery_final_df

In [ ]:
#derive onpromotion under Grocery I

grocery_promo = grocery_sales.groupby('date').onpromotion.sum()
grocery_promo

In [ ]:
nbr_gpromo = pd.concat([grocery_promo, nbr], axis=1)
nbr_gpromo

In [ ]:
nbr_gpromo['Ave Store Promo'] = nbr_gpromo['onpromotion']/nbr_gpromo['store_nbr']
nbr_gpromo

In [ ]:
ave_nbr_gpromo = nbr_gpromo["Ave Store Promo"]
ave_nbr_gpromo

In [ ]:
ave_nbr_gpromo.to_frame()

In [ ]:
grocery_final_df = grocery_final_df.merge(ave_nbr_gpromo, on = 'date', how = 'left')
grocery_final_df



In [ ]:
#Regression Plot: Grocery Sales and Explanatory Variables
plt.figure(figsize=(16,6))
plt.title('Average Grocery Sales per Store and Products on Promotion')
sns.regplot(x=grocery_final_df['Ave Store Promo'], y=grocery_final_df['Ave Store Sales'])
plt.xlabel('Average Grocery Products on Promotion')
plt.ylabel('Average Grocery Sales per Store')

In [ ]:
plt.figure(figsize=(16,6))
plt.title('Average Grocery Sales and Oil Price')
sns.regplot(x=grocery_final_df['dcoilwtico'], y=grocery_final_df['Ave Store Sales'])
plt.xlabel('Oil Price')
plt.ylabel('Average Grocery Sales per Store')

**Diagnostics for Grocery Sales, Promotion**

In [ ]:
# Ave_grocery_sales Diagnostics


ave_nbr_gsales = norm.rvs(size=1000)
sm.qqplot(ave_nbr_gsales, line='45')
pylab.show()

ave_nbr_gsales = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(ave_nbr_gsales)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
# ave_grocery_promo Diagnostics


ave_nbr_gpromo = norm.rvs(size=1000)
sm.qqplot(ave_nbr_gpromo, line='45')
pylab.show()

ave_nbr_gpromo = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(ave_nbr_gpromo)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
#Exploring the correlation of numerical features
plt.figure(figsize=(5,3), dpi=150)
sns.heatmap(grocery_final_df.corr(), annot=True)

In [ ]:
plt.figure(figsize=(25,13))
g = sns.PairGrid(grocery_final_df)
g.map(sns.scatterplot)

# **Regression Analysis by OLS, Random Forest, and Light GBM**

# **First: OLS Regression with Average Daily Sales (All products)**

In [ ]:
#Regression OLS -- Average Total Sales and explanatory variables

# Step 1: Create X and Y data matrices
X = final_df.drop(['Ave Store Sales'], axis = 1)
Y = final_df.drop(['dcoilwtico', 'Ave Store Promo', 'Dummy Holiday'], axis = 1)

In [ ]:
Y

In [ ]:
X

In [ ]:
Y = pd.DataFrame(Y)
Y

In [ ]:
X = pd.DataFrame(X)
X

In [ ]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
X = sm.add_constant(X)
X

In [ ]:
# print regression result
# Average Sales per Store

est = sm.OLS(Y.astype(float), X.astype(float)).fit()
est
print(est.summary())

In [ ]:
est.resid

In [ ]:
# check error normal distribution

est.resid = norm.rvs(size=1000)
sm.qqplot(est.resid, line='45')
pylab.show()

est.resid = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(est.resid)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

# **Second: OLS Regression with Average Grocery Sales**

In [ ]:
# OLS Regression for Ave Grocery Sales

Y_grocery = grocery_final_df.drop(['dcoilwtico', 'Ave Store Promo', 'Dummy Holiday'], axis = 1)
Y_grocery

In [ ]:
X_grocery = grocery_final_df.drop(['Ave Store Sales'], axis = 1)
X_grocery

In [ ]:
X_grocery = sm.add_constant(X_grocery)
X_grocery

In [ ]:
#Regression result for Grocery Sales

model_g = sm.OLS(Y_grocery.astype(float), X_grocery.astype(float))
result = model_g.fit()
print(result.summary())

In [ ]:
# check normal error distribution

result.resid = norm.rvs(size=1000)
sm.qqplot(result.resid, line='45')
pylab.show()

result.resid = np.random.normal(loc = 20, scale = 5, size=150)
statistic,pvalue = lilliefors(result.resid)
print('statistic=%.3f, p=%.3f\n' %  (statistic, pvalue))
if pvalue > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

# **Third: Ave Sales per Store Linear Regression with Sklearn**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Perfrom 80/20 data split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# Step 4: Check data dimension

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


In [ ]:
#regression with sklearn

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print(regressor.intercept_)
print(regressor.coef_)



In [ ]:
y_pred2 = regressor.predict(X_test)
y_pred2

In [ ]:
Y_test

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred2))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred2)))
print('R2:', metrics.r2_score(Y_test, y_pred2))

In [ ]:
Y_test['Predicted Values'] = y_pred2
Y_test

In [ ]:
##### test only #####

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
est2 = sm.OLS(Y_train.astype(float), X_train.astype(float)).fit()
est2
print(est2.summary())

# **Fourth: Random Forest Regression with Ave Daily Sales (all products)**

In [ ]:
# Perfrom 80/20 data split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# Random Forest for Regression

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, Y_train)
y_pred = regressor.predict(X_test)

In [ ]:
Y_test

In [ ]:
y_pred

In [ ]:
print(y_pred.shape)

**Print Metrics of Random Forest Regression**

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
print('R2:', metrics.r2_score(Y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(Y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

In [ ]:
Y_test_copy = Y_test
df_pred = Y_test_copy
df_pred = df_pred.rename(columns={'strength' : 'Actual Values'})
df_pred

In [ ]:
df_pred['Predicted Values'] = y_pred
df_pred['% Difference'] = abs(df_pred['Predicted Values'] - df_pred['Ave Store Sales'])/df_pred['Ave Store Sales']*100
df_pred

# **Fifth: Light GBM with Ave Sales**

In [ ]:
# LGBM

from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as ltb

In [ ]:
# Step 3: Perfrom 70/30 data split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# Step 2 - Using LightGBM Regressor and calculating the scores

model = ltb.LGBMRegressor()
model.fit(X_train.astype(float), Y_train.astype(float))
print(); print(model)
    
expected_y  = Y_test.astype(float)
predicted_y = model.predict(X_test.astype(float))

In [ ]:
expected_y

In [ ]:
print(predicted_y.shape)
predicted_y

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(expected_y, predicted_y))
print('Mean Squared Error:', metrics.mean_squared_error(expected_y, predicted_y))
print('R2:', metrics.r2_score(expected_y, predicted_y))

In [ ]:
df_lgbm = expected_y
df_lgbm['Predicted Values'] = predicted_y
df_lgbm['% Difference'] = abs(df_lgbm['Predicted Values'] - df_lgbm['Ave Store Sales'])/df_lgbm['Ave Store Sales']*100

df_lgbm

In [ ]:
#cross validation but for X_train and Y_train

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

model_linear = LinearRegression()
score = cross_val_score(model_linear, X_train, Y_train, cv=5,n_jobs=-1, scoring='r2')
print(score)

print(score.mean())

# **Sixth: Random Forest Regression with Average Grocery Sales**

In [ ]:
# Random Forest for Grocery Sales
# Step 3: Perfrom 80/20 data split

X_grocery_train, X_grocery_test, Y_grocery_train, Y_grocery_test = train_test_split(X_grocery, Y_grocery, test_size = 0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_grocery_train = sc.fit_transform(X_grocery_train)
X_grocery_test = sc.transform(X_grocery_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=200, random_state=100)
regressor.fit(X_grocery_train, Y_grocery_train)
y_grocery_pred = regressor.predict(X_grocery_test)

In [ ]:
y_grocery_pred

In [ ]:
print(y_grocery_pred.shape)

In [ ]:
Y_grocery_test

Print the Metrics of Random Forest Regression

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(Y_grocery_test, y_grocery_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_grocery_test, y_grocery_pred))
print('R2:', metrics.r2_score(Y_grocery_test, y_grocery_pred))
print('Explained Variance:', metrics.explained_variance_score(Y_grocery_test, y_grocery_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_grocery_test, y_grocery_pred)))

In [ ]:
df_grf = Y_grocery_test
df_grf['Predicted Values'] = y_grocery_pred
df_grf['% Difference'] = abs(df_grf['Predicted Values'] - df_grf['Ave Store Sales'])/df_grf['Ave Store Sales']*100
df_grf

# **Seventh: Light GBM with Average Grocery Sales**

In [ ]:
# Random Forest for Grocery Sales
# Step 3: Perfrom 80/20 data split

X_grocery_train, X_grocery_test, Y_grocery_train, Y_grocery_test = train_test_split(X_grocery, Y_grocery, test_size = 0.2)

In [ ]:
model = ltb.LGBMRegressor()
model.fit(X_grocery_train.astype(float), Y_grocery_train.astype(float))
print(); print(model)
    
expected_g_y  = Y_grocery_test.astype(float)
predicted_g_y = model.predict(X_grocery_test.astype(float))

In [ ]:
expected_g_y

In [ ]:
predicted_g_y

In [ ]:
print('R2:', metrics.r2_score(expected_g_y, predicted_g_y))
print('Mean Squared Error:', metrics.mean_squared_log_error(expected_g_y, predicted_g_y))
print('Mean Absolute Error:', metrics.mean_absolute_error(expected_g_y, predicted_g_y))

In [ ]:
df_glgbm = expected_g_y
df_glgbm['Predicted Values'] = predicted_g_y
df_glgbm['% Difference'] = abs(df_glgbm['Predicted Values'] - df_glgbm['Ave Store Sales'])/df_glgbm['Ave Store Sales']*100

df_glgbm

# **Eighth: Ave Grovery Sales Regression with Sklearn**

In [ ]:
#regression with sklearn

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_grocery_train, Y_grocery_train)
print(regressor.intercept_)
print(regressor.coef_)

In [ ]:
y_gpred2 = regressor.predict(X_grocery_test)
y_gpred2

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_grocery_test, y_gpred2))
print('Mean Squared Error:', metrics.mean_squared_error(Y_grocery_test, y_gpred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_grocery_test, y_gpred2)))
print('R2:', metrics.r2_score(Y_grocery_test, y_gpred2))